In [ ]:
import numpy as np 
import pandas as pd 
import pandas_profiling as pp
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
%matplotlib inline

import seaborn as sns
sns.set()

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
df = pd.read_csv('../input/ecommerce-data/data.csv')
df.head()

In [ ]:
df.describe()

A lot of Outliners, NaN's in CustomerID, negative values

**Period of data**

In [ ]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df.InvoiceDate.max() - df.InvoiceDate.min()

In [ ]:
format(df.InvoiceDate.min())

In [ ]:
format(df.InvoiceDate.max())

373 days - 2010-12-01 - 2011-12-09

**Outliners**

Quantity

In [ ]:
plt.figure(figsize=(18,6))
plt.scatter(df.InvoiceDate, df.Quantity)

Price

In [ ]:
plt.figure(figsize=(18,6))
plt.scatter(df.InvoiceDate, df.UnitPrice)

Data gapes 2011-01, 2011-05 

A lot of outliners, negative values

**Missing data**

In [ ]:
df.isnull().sum()

Missing data %

In [ ]:
df.isnull().sum() / df.shape[0] * 100

25% missing data - CustomerID
0,3% - Description

**Missing Descriptions**

In [ ]:
df[df.Description.isnull()].head(5)

Description = NaN, Unitprice = 0, CustomerID = NaN ?

**Missing Customer ID and UnitPrice**

In [ ]:
df[df.Description.isnull()].CustomerID.isnull().value_counts()

In [ ]:
df[df.Description.isnull()].UnitPrice.value_counts()

In [ ]:
df[df.CustomerID.isnull()].head()

In [ ]:
df = df.loc[(df.CustomerID.isnull()==False)].copy()
df.isnull().sum()

**Invoices - Deleting 'C'**

In [ ]:
df["C"]=np.where(df.InvoiceNo.apply(lambda l: l[0]=="C"), True, False)
df.C.value_counts() / df.shape[0] * 100

In [ ]:
df.loc[df["C"]].describe()

In [ ]:
df = df.loc[df.C==False].copy()
df = df.drop("C", axis=1)

**StockCodes - Bad Values**

In [ ]:
plt.figure(figsize=(15,6))
TopStocks = df['StockCode'].value_counts().sort_values(ascending=False).iloc[0:10]
sns.barplot(TopStocks.index, TopStocks.values)

POST

In [ ]:
def CountChars(l):
    return sum(1 for c in l if c.isdigit())

df["LengthOfStockCode"] = df.StockCode.apply(lambda l: len(l))
df["NumbersInStockCode"] = df.StockCode.apply(lambda l: CountChars(l))

In [ ]:
ax=sns.countplot(df["LengthOfStockCode"])
ax.set_yscale("log")

In [ ]:
ax=sns.countplot(df["NumbersInStockCode"])
ax.set_yscale("log")

In [ ]:
df.loc[df.NumbersInStockCode < 5].Description.value_counts()

In [ ]:
df = df.loc[(df.NumbersInStockCode == 5) & (df.LengthOfStockCode==5)].copy()

**Duplicates in StockCodes**

In [ ]:
df.groupby("StockCode").Description.nunique().sort_values(ascending=False).head(10)

In [ ]:
df.loc[df.StockCode == "23236"].Description.value_counts()

The same products

**Descriptions - Low Chars**

In [ ]:
def LowChars(l):
    return sum(1 for c in l if c.islower())

In [ ]:
df["LengthOfDescription"] = df.Description.apply(lambda l: len(l))
df["DescriptionLowChars"] = df.Description.apply(lambda l: LowChars(l))

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.countplot(df.LengthOfDescription)
ax.set_yscale("log")

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.countplot(df.DescriptionLowChars)
ax.set_yscale("log")

In [ ]:
df.loc[df.DescriptionLowChars > 0].Description.value_counts()

In [ ]:
df.loc[df.DescriptionLowChars > 2].Description.value_counts()

In [ ]:
df = df.loc[df.DescriptionLowChars < 3].copy()

In [ ]:
df = df.drop(["LengthOfStockCode","NumbersInStockCode","LengthOfDescription","DescriptionLowChars"], axis=1)

**Unit Prices**

In [ ]:
df.UnitPrice.describe()

Free?

In [ ]:
df.loc[df.UnitPrice == 0].sort_values(by="Quantity", ascending=False).head()

In [ ]:
df = df.loc[df.UnitPrice > 0].copy()

In [ ]:
df.UnitPrice.describe()

In [ ]:
plt.figure(figsize=(18,6))
plt.scatter(df.InvoiceDate, df.UnitPrice)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.distplot(np.log(df.UnitPrice),bins=20,kde=False)
ax.set_xlabel("Log Unit-Price");

In [ ]:
np.exp(-2)

In [ ]:
np.exp(3)

In [ ]:
np.quantile(df.UnitPrice, 0.95)

In [ ]:
df = df.loc[(df.UnitPrice > 0.1) & (df.UnitPrice < 20)].copy()

In [ ]:
plt.figure(figsize=(18,6))
plt.scatter(df.InvoiceDate, df.UnitPrice)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.distplot(np.log(df.UnitPrice),bins=20,kde=False)
ax.set_xlabel("Log Unit-Price");

**Quantity**

In [ ]:
df.Quantity.describe()

In [ ]:
plt.figure(figsize=(18,6))
plt.scatter(df.InvoiceDate, df.Quantity)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.distplot(np.log(df.Quantity),bins=20,kde=False)
ax.set_xlabel("Log Quantity");

In [ ]:
np.exp(4)

In [ ]:
np.quantile(df.Quantity, 0.95)

In [ ]:
df = df.loc[df.Quantity < 55].copy()

In [ ]:
plt.figure(figsize=(18,6))
plt.scatter(df.InvoiceDate, df.Quantity)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.distplot(np.log(df.Quantity),bins=20,kde=False)
ax.set_xlabel("Log Quantity");

# End of Cleaning the Data

**Countries**

In [ ]:
countries = df.Country.value_counts().sort_values(ascending=False).iloc[0:20]
plt.figure(figsize=(15,5))
sns.barplot(countries.index, countries.values, palette="Purples_r")
plt.xticks(rotation=90);
plt.yscale("log")

In [ ]:
df.loc[df.Country=="United Kingdom"].shape[0] / df.shape[0] * 100

In [ ]:
data = dict(type='choropleth',
locations = countries.index,
locationmode = 'country names', z = countries,
text = countries.index, colorbar = {'title':'Count.'},
colorscale=[[0, 'rgb(224,255,255)'],
            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],
            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],
            [1, 'rgb(227,26,28)']],    
reversescale = False)
layout = dict(title='Map',
geo = dict(showframe = True, projection={'type':'equirectangular'}))
choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate=False)

**Additional columns**

In [ ]:
df['Sales']=df['Quantity']*df['UnitPrice']

In [ ]:
periods=['year','month','day','dayofweek','dayofyear','weekofyear','quarter']
df=df.join(pd.concat((getattr(df['InvoiceDate'].dt,i).rename(i) for i in periods),axis=1))
df.insert(loc=2,column='year_month',value=df['InvoiceDate'].map(lambda x: 100*x.year + x.month))
df.insert(loc=5,column='hour',value=df.InvoiceDate.dt.hour)

In [ ]:
ax=df.groupby('InvoiceNo')['year_month'].unique().value_counts().sort_index().plot(kind='bar',figsize=(15,6))
ax.set_xticklabels(('Dec','Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sep','Oct','Nov','Dec'));

In [ ]:
ax=df.groupby('InvoiceNo')['quarter'].unique().value_counts().sort_index().plot(kind='bar',figsize=(20,6))

In [ ]:
ax=df.groupby('InvoiceNo')['weekofyear'].unique().value_counts().sort_index().plot(kind='bar',figsize=(20,6))

In [ ]:
ax=df.groupby('InvoiceNo')['day'].unique().value_counts().sort_index().plot(kind='bar',figsize=(20,6))

In [ ]:
df['Description'].value_counts().sort_values(ascending=False).head(10)

In [ ]:
fig, ax=plt.subplots(figsize=(10,5))
df['Description'].value_counts().sort_values(ascending=False).head(10).plot(kind='barh')

In [ ]:
df2 = df.pivot_table(index='hour',columns='Description', aggfunc={'Description':'count'}).fillna(0)
df2['Max']=df2.idxmax(axis=1)
df2['Max']

# **RFM**

In [ ]:
import datetime as dt 
now = dt.date(2011,12,9)
df['date'] = pd.DatetimeIndex(df.InvoiceDate).date

In [ ]:
df['CustomerID']=df['CustomerID'].astype('int64')

In [ ]:
recency_df = df.groupby(['CustomerID'],as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df['Recency'] = recency_df.LastPurchaseDate.apply(lambda x : (now - x).days)
recency_df.drop(columns=['LastPurchaseDate'],inplace=True)
recency_df.head()

In [ ]:
frequency_df = df.copy()
frequency_df.drop_duplicates(subset=['CustomerID','InvoiceNo'], keep="first", inplace=True) 
frequency_df = frequency_df.groupby('CustomerID',as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

In [ ]:
monetary_df=df.groupby('CustomerID',as_index=False)['Sales'].sum()
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

In [ ]:
rf = recency_df.merge(frequency_df,left_on='CustomerID',right_on='CustomerID')
rfm = rf.merge(monetary_df,left_on='CustomerID',right_on='CustomerID')
rfm.set_index('CustomerID',inplace=True)
rfm.head()

In [ ]:
(now - dt.date(2011,1,18)).days == 325

In [ ]:
rfm_segmentation = rfm.copy()

In [ ]:
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
score = [kmeans[i].fit(rfm_segmentation).score(rfm_segmentation) for i in range(len(kmeans))]
plt.plot(Nc,score)
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(rfm_segmentation)
rfm_segmentation['cluster'] = kmeans.labels_
rfm_segmentation[rfm_segmentation.cluster == 0].head(10)

In [ ]:
sns.boxplot(rfm_segmentation.cluster,rfm_segmentation.Recency)

In [ ]:
sns.boxplot(rfm_segmentation.cluster,rfm_segmentation.Frequency)

In [ ]:
sns.boxplot(rfm_segmentation.cluster,rfm_segmentation.Monetary)

In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
quantile = rfm.quantile(q=[0.25,0.5,0.75])
quantile

In [ ]:
rfm_segmentation['R'] = rfm_segmentation['Recency'].apply(RScore,args=('Recency',quantile))
rfm_segmentation['F'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantile))
rfm_segmentation['M'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantile))

In [ ]:
quantile.to_dict()

In [ ]:
rfm_segmentation['RFM'] = rfm_segmentation.R.map(str) \
                            + rfm_segmentation.F.map(str) \
                            + rfm_segmentation.M.map(str)

# Segmentation

In [ ]:
print("Best: ",len(rfm_segmentation[rfm_segmentation['RFM']=='111']))
print('Loyal: ',len(rfm_segmentation[rfm_segmentation['F']==1]))
print("Spenders: ",len(rfm_segmentation[rfm_segmentation['M']==1]))
print('Almost lost: ', len(rfm_segmentation[rfm_segmentation['RFM']=='134']))
print('Lost: ',len(rfm_segmentation[rfm_segmentation['RFM']=='344']))
print('Lost but not spenders: ',len(rfm_segmentation[rfm_segmentation['RFM']=='444']))

In [ ]:
rfm_segmentation[rfm_segmentation['RFM'] == '111'].sort_values('Monetary', ascending=False).head(3)

# Bucket Association Rules Apriori

In [ ]:
df["Description"].nunique()
data_apriori = df.copy()
data_apr = data_apriori.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
data_apr.head()

In [ ]:
def num(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_new = data_apr.applymap(num)
basket_new.head()

In [ ]:
items = apriori(basket_new, min_support=0.02, use_colnames=True)
items.head()

In [ ]:
rule = association_rules(items, metric="lift", min_threshold=1)
rule.sort_values(by=['confidence'],ascending=False).head(10)